In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install imblearn
%pip install numpy
%pip install matplotlib

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('./dataset/Amazon_Product_Review_cleaned.csv')

# Hiển thị các cột của dataset (để kiểm tra)
print("Các cột trong dataset:", data.columns.tolist())

# 2. Tiền xử lý văn bản trong cột 'review'
def preprocess_text(text):
    text = str(text)            # Đảm bảo text là chuỗi
    text = text.lower()         # Chuyển về chữ thường
    # Giữ lại các ký tự chữ cái (bao gồm có dấu), số và dấu cách
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

# Áp dụng tiền xử lý cho cột review
data['review'] = data['review_headline'].apply(preprocess_text)

# 3. Loại bỏ các dòng có giá trị bị thiếu
data = data.dropna(subset=['review', 'star_rating'])

# 4. Đảm bảo cột star_rating ở dạng số nguyên
data['star_rating'] = data['star_rating'].astype(int)

# Hiển thị các giá trị star_rating độc nhất
print("Các star_rating độc nhất:", data['star_rating'].unique())

# 5. Tách dữ liệu thành đặc trưng (X) và nhãn (y)
X = data['review']
y = data['star_rating']

# 6. Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Trích xuất đặc trưng từ văn bản bằng TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# # 9. Áp dụng SMOTE để cân bằng dữ liệu lớp thiểu số
# X_train_dense = X_train_tfidf.toarray()  # Chuyển đổi ma trận thưa thành ma trận đặc
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_dense, y_train)

# 8. Xây dựng và huấn luyện mô hình SVM (phân loại đa lớp: 1,2,3,4,5)
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_tfidf, y_train)
# # 10. Dự đoán nhãn cho tập kiểm tra
# print("Số lượng mẫu trước SMOTE:", Counter(y_train))
# print("Số lượng mẫu sau SMOTE:", Counter(y_train_resampled))

# # Define the pipeline
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(max_features=5000)),
#     ('svm', SVC(kernel='rbf', random_state=42))
# ])

# cv_scores = cross_val_score(pipeline, X, y, cv=2, scoring='accuracy')
# print("Cross-validation accuracy scores:", cv_scores)
# print("Mean CV accuracy:", cv_scores.mean())
# print("Mean CV accuracy:", cv_scores.mean())

In [ ]:
# 9. Dự đoán và đánh giá mô hình
y_pred = svm_model.predict(X_test_tfidf)

print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred), "\n")
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred), "\n")
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))

In [ ]:

# 9. Kiểm thử với các câu thực tế
test_sentences = [

    # 30% tiếng Anh (3/10)
    "Amazing quality, definitely worth it!",
    "Terrible experience, will never buy again.",
    "I am extremely dissatisfied with this product.",
]

X_test_sentences = tfidf.transform(test_sentences)
y_test_pred = svm_model.predict(X_test_sentences)

# Hiển thị kết quả
for text, pred in zip(test_sentences, y_test_pred):
    print(f"Review: {text} -> Dự đoán: {pred} sao")